In [19]:
from fastai.vision.all import *
from fastbook import *
matplotlib.rc('image', cmap='Greys')

In [235]:
def batch_accuracy(xb, yb):
    predictions = xb.sigmoid()
    correct = (predictions > 0.5) == yb
    return correct.float().mean()

In [236]:
def mnist_loss(predictions, target):
    predictions = predictions.sigmoid()
    return torch.where(target==1,1-predictions,predictions).mean()

In [237]:
path = untar_data(URLs.MNIST_SAMPLE)

In [238]:
path.ls()

(#3) [Path('C:/Users/antho/.fastai/data/mnist_sample/labels.csv'),Path('C:/Users/antho/.fastai/data/mnist_sample/train'),Path('C:/Users/antho/.fastai/data/mnist_sample/valid')]

In [239]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

In [240]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors),len(seven_tensors)

(6131, 6265)

In [241]:
stacked_threes = torch.stack(three_tensors).float()/255
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes.shape

torch.Size([6131, 28, 28])

In [242]:
valid3_tens = torch.stack([tensor(Image.open(o)) 
for o in (path/'valid'/'3').ls()])
valid3_tens = valid3_tens.float()/255

valid7_tens = torch.stack([tensor(Image.open(o))
for o in (path/'valid'/'7').ls()])
valid7_tens = valid7_tens.float()/255

valid3_tens.shape, valid7_tens.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [243]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
train_x.shape, train_y.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [244]:
dset = list(zip(train_x, train_y))
x,y = dset[0]
x.shape, y.shape

(torch.Size([784]), torch.Size([1]))

In [245]:
valid_x = torch.cat([valid3_tens, valid7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid3_tens) + [0]*len(valid7_tens)).unsqueeze(1)
validDset = list(zip(valid_x, valid_y))

In [246]:
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(validDset, batch_size=256)

In [247]:
dls = DataLoaders(dl, valid_dl)

In [258]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1),
)

In [259]:
learn = Learner(dls, simple_net, opt_func=SGD, 
                loss_func=mnist_loss, metrics=accuracy)

In [260]:
learn.fit(10, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.279214,0.407621,0.504416,00:00
1,0.133332,0.219950,0.504416,00:00
2,0.076054,0.113934,0.504416,00:00
3,0.051352,0.077786,0.504416,00:00
4,0.039686,0.060980,0.504416,00:00
5,0.033589,0.051481,0.504416,00:00
6,0.029996,0.045463,0.504416,00:00
7,0.027609,0.041320,0.504416,00:00
8,0.025860,0.038289,0.504416,00:00
9,0.024491,0.035961,0.504416,00:00


In [255]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1),
)

In [256]:
check = Learner(dls, simple_net, opt_func=SGD, 
                loss_func=mnist_loss, metrics=batch_accuracy)

In [257]:
check.fit(10, 0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.359944,0.394128,0.521590,00:00
1,0.161579,0.245561,0.783121,00:00
2,0.087561,0.121759,0.907262,00:00
3,0.056168,0.080865,0.940137,00:00
4,0.041779,0.062536,0.952404,00:00
5,0.034552,0.052322,0.962218,00:00
6,0.030478,0.045918,0.965653,00:00
7,0.027879,0.041560,0.966143,00:00
8,0.026028,0.038404,0.968597,00:00
9,0.024607,0.035996,0.971050,00:00


In [273]:
path = untar_data(URLs.MNIST)

In [274]:
path.ls()

(#2) [Path('C:/Users/antho/.fastai/data/mnist_png/testing'),Path('C:/Users/antho/.fastai/data/mnist_png/training')]

**Counting total images**

In [275]:
totalTrain = 0
totalTest = 0

for i in range(10):
    totalTrain += len((path/'training'/f'{i}').ls().sorted())
    totalTest += len((path/'testing'/f'{i}').ls().sorted())

print(f'Total Train: {totalTrain}') 
print(f'Total Test: {totalTest}') 

Total Train: 60000
Total Test: 10000


**Assembling to a list**

In [276]:
training = []
testing = []
for i in range(10):
    training.extend((path/'training'/f'{i}').ls().sorted())
    testing.extend((path/'testing'/f'{i}').ls().sorted())

In [277]:
len(training), len(testing)

(60000, 10000)

In [278]:
numbersTrain = [tensor(Image.open(o)) for o in training]
numbersTest = [tensor(Image.open(o)) for o in testing]

In [279]:
numbersTrain[1].shape, numbersTest[1].shape

(torch.Size([28, 28]), torch.Size([28, 28]))

In [280]:
stackTrain = torch.stack(numbersTrain).float()/255
stackTest = torch.stack(numbersTest).float()/255
stackTrain.shape, stackTest.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [328]:
trainLabel = []
for i in range(10):
    trainLabel.extend([i]*len((path/'training'/f'{i}').ls().sorted()))
len(trainLabel), trainLabel[20000]

(60000, 3)

In [329]:
testLabel = []
for i in range(10):
    testLabel.extend([i]*len((path/'testing'/f'{i}').ls().sorted()))
len(testLabel), testLabel[1000]

(10000, 1)

In [330]:
train_x = stackTrain.view(-1, 28*28)
train_y = tensor(trainLabel)
train_x.shape, train_y.shape

(torch.Size([60000, 784]), torch.Size([60000]))

In [331]:
test_x = stackTest.view(-1, 28*28)
test_y = tensor(testLabel)
test_x.shape, test_y.shape

(torch.Size([10000, 784]), torch.Size([10000]))

In [332]:
dset = list(zip(train_x, train_y))
x,y = dset[0]
x.shape, y.shape

(torch.Size([784]), torch.Size([]))

In [333]:
valid_dset = list(zip(test_x, test_y))
x,y = dset[0]
x.shape, y.shape

(torch.Size([784]), torch.Size([]))

In [334]:
train_dl = DataLoader(train_dset, batch_size=256)
test_dl = DataLoader(test_dset, batch_size=256)

In [335]:
xb,yb = first(train_dl)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

In [336]:
xb,yb = first(test_dl)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

In [337]:
def mnist_loss(predictions, target):
    predictions = torch.softmax(predictions, dim=1)
    loss = torch.nn.functional.cross_entropy(predictions, target)
    return loss

In [338]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,10),
    nn.Softmax(dim=1)
)

In [339]:
dls = DataLoaders(train_dl, test_dl)

In [340]:
learn = Learner(dls, simple_net, opt_func=SGD, 
                loss_func=mnist_loss, metrics=batch_accuracy)

In [341]:
learn.fit(5,0.1)

epoch,train_loss,valid_loss,batch_accuracy,time


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [327]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,10),
    nn.Softmax(dim=1)
)

In [306]:
learn = Learner(dls, simple_net, opt_func=SGD, 
                loss_func=mnist_loss, metrics=accuracy)

In [307]:
learn.fit(5,0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.475022,0.519308,0.090800,00:00
1,0.475022,0.519308,0.090800,00:00
2,0.475022,0.519308,0.090800,00:00
3,0.475022,0.519308,0.090800,00:00
4,0.475022,0.519308,0.090800,00:00
